In [7]:
import pdb
import h5py
import glob
import numpy as np
from astropy.table import Table
from Payne.fitting.fitutils import airtovacuum
from astropy.time import Time
from astropy.coordinates import SkyCoord, EarthLocation
import astropy.units as u
from scipy import constants
import os
# from runscripts.runUTPbinary_25010_1 import getdata as my_getdata

speedoflight = constants.c / 1000.0

In [100]:
def getdata(catfile=None,gaiaid=None,starind=None):
    # h5 file
    th5 = h5py.File(catfile,'r')

    # put h5 file in an astropy table
    cat_i = Table(th5['catalog'][()])
    
    if gaiaid != None:
        # h5 with just gaiaid data
        t = th5[f'{gaiaid}']

        # table with just gaiaid data
        cat_ii = cat_i[cat_i['GAIAEDR3_ID'] == gaiaid]

        # phot dict
        cat = {x:cat_ii[x][0] for x in cat_ii.keys()}

    elif starind != None:
        cat_ii = cat_i[starind]
        cat = {x:cat_ii[x] for x in cat_ii.keys()}
        
        gaiaid = cat['GAIAEDR3_ID']
        t = th5[f'{gaiaid}']


    # print(f"t: {t['wave'][()]}")
    header = {ii:th5['header'][ii][()] for ii in th5['header'].keys()}
    hdr_date = header['DATE-OBS']
    location = EarthLocation.of_site('MMT')

    sc = SkyCoord(ra=cat['GAIAEDR3_RA']*u.deg,dec=cat['GAIAEDR3_DEC']*u.deg)
    barycorr = sc.radial_velocity_correction(obstime=Time(hdr_date), location=location)
    HC = float(barycorr.to(u.km/u.s).value)
    
    out = {}
    out['spec'] = {}
    out['phot'] = {}
    
    # spec
    out['spec']['obs_wave']  = (1.0 - (HC/speedoflight)) * airtovacuum(t['wave'][()])
    out['spec']['obs_flux']  = t['flux'][()]
    out['spec']['obs_eflux'] = t['eflux'][()]

    # medflux = np.nanmedian(out['spec']['obs_flux'])
    # out['spec']['obs_flux'] = out['spec']['obs_flux'] / medflux
    # out['spec']['obs_eflux'] = out['spec']['obs_eflux'] / medflux

    SNR = np.nanmedian(out['spec']['obs_flux']/out['spec']['obs_eflux'])
    out['spec']['SNR'] = SNR
    out['spec']['date'] = hdr_date
    out['spec']['HC'] = HC

    # phot
    phot = t['phot']
    filterarr = list(phot.keys())
    usedfilters = []
    
    for ff in filterarr:            
        phot_i = phot[ff][()]
        if np.isfinite(phot_i[0]) & (phot_i[1] > 0.0):
            if 'Gaia' in ff:
                ff = ff.replace('EDR3', 'DR3')
            if 'PS_y' in ff:
                print(f"Skipping filter {ff}")
                continue
            out['phot'][ff] = [phot_i[0],phot_i[1]]
            usedfilters.append(ff)
    
    out['phot_filtarr'] = usedfilters
    # print("phot filtarr followed by phot")
    # print(out['phot_filtarr'])
    # print()
    # print(out['phot'])
    
    # parallax
    out['parallax'] = [cat['GAIAEDR3_PARALLAX'],cat['GAIAEDR3_PARALLAX_ERROR']]
    
    # define some cluster guesses
    if 'ic348' in catfile:
        Avest = 1.91 # Cantat-Gaudin+ 2020
        RVest = 15.44 # Tarricq+ 2021
    elif 'm37' in catfile:
        Avest = 0.75
        RVest = 8.81
    elif 'm3' in catfile:
        Avest = 0.03 # Harris 1996
        RVest = -147.2 # BAUMGARDT H. and HILKER M 2018
    elif 'm67' in catfile:
        Avest = 0.07
        RVest = 34.18
    elif 'ngc6791' in catfile:
        Avest = 0.70
        RVest = -47.75
    elif 'ngc6811' in catfile:
        Avest = 0.09
        RVest = 7.17
    elif 'ngc6819' in catfile:
        Avest = 0.40
        RVest = 2.80
    elif 'ngc6866' in catfile:
        Avest = 0.48
        RVest = 12.44
    else:
        print('Could not understand cluster name in filename, setting Avest = 0.1 and RVest = 0.0')
        Avest = 0.1
        RVest = 0.0
    
    out['Avest'] = Avest
    out['RVest'] = RVest

    out['Teff']   = 5770.0
    out['[Fe/H]'] = 0.0
    out['[a/Fe]'] = 0.0
    out['log(g)'] = 4.5
    out['log(R)'] = 0.0

    return out

In [102]:
gaiaid=2076394173950855424
# data = getdata(catfile='../data/hectochelle_rereduction/data_ngc6819_2012.0509_ngc6819_may2012_3.0223.h5', gaiaid=gaiaid)
data = getall(gaiaid=gaiaid, cluster='ngc6819')
#data
data.keys()
data['phot']

['/data/labs/douglaslab/sofairj/data/hectochelle_rereduction/data_ngc6819_2012.0509_ngc6819_may2012_3.0223.h5']
Skipping filter PS_y
out phot filtarr in get all: ['2MASS_H', '2MASS_J', 'GaiaDR3_BP', 'GaiaDR3_G', 'GaiaDR3_RP', 'PS_g', 'PS_i', 'PS_r']
data phot filtarr in get all: ['2MASS_H', '2MASS_J', 'GaiaDR3_BP', 'GaiaDR3_G', 'GaiaDR3_RP', 'PS_g', 'PS_i', 'PS_r']


{'2MASS_H': [13.654999732971191, 0.0679411510663857],
 '2MASS_J': [13.940999984741211, 0.060464866870328884],
 'GaiaDR3_BP': [15.423774144665938, 0.05002172265077965],
 'GaiaDR3_G': [15.084918, 0.05000058661244882],
 'GaiaDR3_RP': [14.57160009952022, 0.05001403533170928],
 'PS_g': [15.46501350402832, 0.020529308063029533],
 'PS_i': [14.930351257324219, 0.020429329605116866],
 'PS_r': [15.064974784851074, 0.020432112800901934]}

In [42]:
def getall(gaiaid=None,date=None,cluster=None):
    
    datasrcstr = '/data/labs/douglaslab/sofairj/data/hectochelle_rereduction/'
    
    filesrstr = '*'
    # list of all possible HDF5 files
    if cluster != None:
        filesrstr += f'{cluster}*'
    if date != None:
        filesrstr += f'{date}*'

    filesrstr += '.h5'
    flist = glob.glob(datasrcstr+filesrstr)

    # get the HDF5 files with this star
    workingfiles = []
    for ff in flist:
        with h5py.File(ff,'r') as th5:
            if f'{gaiaid}' in list(th5.keys()):
                workingfiles.append(ff)    
    if len(workingfiles) == 0:
        print(f'Could Not Find Any Spectra for GaiaID = {gaiaid}')
        return None

    print(workingfiles)
    
    out = {}
    out['spec'] = []
    out['specname'] = []
    for ii,ww in enumerate(workingfiles):
        data = getdata(catfile=ww,gaiaid=gaiaid)
        if ii == 0:
            out['phot']     = data['phot']
            out['phot_filtarr'] = data['phot_filtarr']
            out['parallax'] = data['parallax']
            out['Avest']    = data['Avest']
            out['RVest']    = data['RVest']

            out['Teff']   = data['Teff']
            out['[Fe/H]'] = data['[Fe/H]']
            out['[a/Fe]'] = data['[a/Fe]']
            out['log(g)'] = data['log(g)']
            out['log(R)'] = data['log(R)']

        if data['spec']['SNR'] >= 3.0:
            out['spec'].append(data['spec'])
            out['specname'].append(ww.split('/')[-1])
    
    if len(out['spec']) > 2:
        # sort spectra by date
        sortind = np.argsort(out['specname'])
        out['spec'] = list(np.array(out['spec'])[sortind])
        out['specname'] = list(np.array(out['specname'])[sortind])

    print(f"out phot filtarr in get all: {out['phot_filtarr']}")
    print(f"data phot filtarr in get all: {data['phot_filtarr']}")

    return out

In [43]:
gaiaid=2076394173950855424
# data = getdata(catfile='../data/hectochelle_rereduction/data_ngc6819_2012.0509_ngc6819_may2012_3.0223.h5', gaiaid=gaiaid)
data = getall(gaiaid=gaiaid, cluster='ngc6819')
#data
#data['spec']

['/data/labs/douglaslab/sofairj/data/hectochelle_rereduction/data_ngc6819_2012.0509_ngc6819_may2012_3.0223.h5']
phot filtarr followed by phot
['2MASS_H', '2MASS_J', 'GaiaDR3_BP', 'GaiaDR3_G', 'GaiaDR3_RP', 'PS_g', 'PS_i', 'PS_r', 'PS_y']

{'2MASS_H': [13.654999732971191, 0.0679411510663857], '2MASS_J': [13.940999984741211, 0.060464866870328884], 'GaiaDR3_BP': [15.423774144665938, 0.05002172265077965], 'GaiaDR3_G': [15.084918, 0.05000058661244882], 'GaiaDR3_RP': [14.57160009952022, 0.05001403533170928], 'PS_g': [15.46501350402832, 0.020529308063029533], 'PS_i': [14.930351257324219, 0.020429329605116866], 'PS_r': [15.064974784851074, 0.020432112800901934], 'PS_y': [14.845098495483398, 0.020433520544655397]}
out phot filtarr in get all: ['2MASS_H', '2MASS_J', 'GaiaDR3_BP', 'GaiaDR3_G', 'GaiaDR3_RP', 'PS_g', 'PS_i', 'PS_r', 'PS_y']
data phot filtarr in get all: ['2MASS_H', '2MASS_J', 'GaiaDR3_BP', 'GaiaDR3_G', 'GaiaDR3_RP', 'PS_g', 'PS_i', 'PS_r', 'PS_y']


In [68]:
def my_getdata(gaiaid='', datasrcstr=''):
    # read in observed spectrum
    hecto_dir = os.path.expanduser("/data/labs/douglaslab/sofairj/data/hectochelle_rereduction")
    hecto_filename = os.path.join(hecto_dir, "data_ngc6819_2012.0509_ngc6819_may2012_3.0223.h5")
    f = h5py.File(hecto_filename, 'r')

    target = str(2076394173950855424)
    spec = Table([f[target]['wave'], f[target]['flux'],
                  f[target]['eflux']],
                  names=('wave', 'flux', 'eflux'))
    spec = spec[(spec['wave'] > 5150.0) & (spec['wave'] < 5300.0)]
    # breakpoint()
    # read in phot
    # create table with photometry for every filter
    phottab = f[target]['phot']

    # get the filters
    filtarr = phottab.keys()

    phot = {}
    # create a dict with {filter name: [flux magnitude, flux error]}
    for i, filter in enumerate(filtarr):
        # skip the PS_y filter because the NN is not trained on it??
        # it is at least not being generated by the genphot function in runSVI
        if filter != 'PS_y':
            phot[filter] = [float(phottab[filter][0]),float(phottab[filter][1])]
        else:
            print(f'Skipping {filter} filter')
        # phot[filter] = [float(phottab[filter][0]),float(phottab[filter][1])]

    # Need to change the names of the Gaia EDR3 filters to DR3
    # These filters are unchanged between EDR3 and DR3
    phot['GaiaDR3_BP'] = phot.pop('GaiaEDR3_BP')
    phot['GaiaDR3_RP'] = phot.pop('GaiaEDR3_RP')
    phot['GaiaDR3_G'] = phot.pop('GaiaEDR3_G')

    # read in MIST isochrone
    # iso = at.read("./data/MIST_iso_67e56fd8ac521.iso.cmd")
    # filter out AGB/RGB stars so we're left with just main sequence stars
    # ms = (iso['EEP'] < 605) & (iso['log_g'] > 2.0)
    # iso = iso[ms]
    #print(f"Here is your iso in the getdata():\n{iso}")
    # create out dict
    out = {}

    out['spec'] = {}
    out['spec']['obs_wave'] = spec['wave']
    # out['spec']['obs_wave']  = (1.0 - (HC/speedoflight)) * airtovacuum(spec['wave'][()])

    out['spec']['obs_flux'] = spec['flux']
    out['spec']['obs_eflux'] = spec['eflux']
    
    out['phot'] = {}
    # for each filter, add the magnitude and error to the out dict
    for kk in phot:
        phot_i = phot[kk]
        out['phot'][kk] = [phot_i[0],phot_i[1]]

    #out['iso'] = iso['log_Teff', 'log_g', 'initial_mass']
    #out['iso']['Teff'] = 10**out['iso']['log_Teff']
    #del(out['iso']['log_Teff'])

    out['parallax'] = [0.3821, 0.0181]
    out['RVest'] = 2.5
    out['Avest'] = 0.09

    # print(f'Just before outputting, here is your out dict:\n{out}')

    return out

In [90]:
gaiaid=2076394173950855424
# data, t = getdata(catfile='../data/hectochelle_rereduction/data_ngc6819_2012.0509_ngc6819_may2012_3.0223.h5', gaiaid=gaiaid)
data = getall(gaiaid=gaiaid, cluster='ngc6819')
data2 = my_getdata()

['/data/labs/douglaslab/sofairj/data/hectochelle_rereduction/data_ngc6819_2012.0509_ngc6819_may2012_3.0223.h5']
out phot filtarr in get all: ['2MASS_H', '2MASS_J', 'GaiaDR3_BP', 'GaiaDR3_G', 'GaiaDR3_RP', 'PS_g', 'PS_i', 'PS_r', 'PS_y']
data phot filtarr in get all: ['2MASS_H', '2MASS_J', 'GaiaDR3_BP', 'GaiaDR3_G', 'GaiaDR3_RP', 'PS_g', 'PS_i', 'PS_r', 'PS_y']
Skipping PS_y filter


In [80]:
t

array([5160.00330021, 5160.03913355, 5160.07496686, ..., 5289.93561784,
       5289.96740831, 5289.99919684], dtype='>f8')

In [91]:
data.keys(), data2.keys()

(dict_keys(['spec', 'specname', 'phot', 'phot_filtarr', 'parallax', 'Avest', 'RVest', 'Teff', '[Fe/H]', '[a/Fe]', 'log(g)', 'log(R)']),
 dict_keys(['spec', 'phot', 'parallax', 'RVest', 'Avest']))

In [95]:
data['spec'][0]['obs_wave'][:5]

array([5161.18824286, 5161.224084  , 5161.2599251 , 5161.29576615,
       5161.33160717])

In [84]:
np.where((t == np.array(data2['spec']['obs_wave'])) == False)
# data['spec'][0]['obs_wave'], data2['spec']['obs_wave']

(array([], dtype=int64),)

In [10]:
data2['phot']

{'2MASS_H': [13.654999732971191, 0.0679411510663857],
 '2MASS_J': [13.940999984741211, 0.060464866870328884],
 'PS_g': [15.46501350402832, 0.020529308063029533],
 'PS_i': [14.930351257324219, 0.020429329605116866],
 'PS_r': [15.064974784851074, 0.020432112800901934],
 'PS_y': [14.845098495483398, 0.020433520544655397],
 'GaiaDR3_BP': [15.423774144665938, 0.05002172265077965],
 'GaiaDR3_RP': [14.57160009952022, 0.05001403533170928],
 'GaiaDR3_G': [15.084918, 0.05000058661244882]}

In [54]:
from astropy.table import QTable, Table, Column
from astropy import units as u
import numpy as np

a = np.array([1, 4], dtype=np.int32)
b = [2.0, 5.0]
c = ['x', 'y']
d = [89, 2.2]
e = [17, 723]
f = ['hi', 'no']
g = ['shorb', 3]

t = Table([a, b, c, d, e, f, g], names=('a', 'b', 'c', 'd', 'e', 'f', 'g'))
filtarr = ['a', 'w', 'b', 'z', 'c', 'e', 'f', 'g', 'd', 'y']
# t = [t[xx] for xx in filtarr]
try:
    t = [t[xx] for xx in filtarr]
except KeyError:
    pass
t

a,b,c,d,e,f,g
int32,float64,str1,float64,int64,str2,str21
1,2.0,x,89.0,17,hi,shorb
4,5.0,y,2.2,723,no,3


In [60]:
th5 = h5py.File('../data/hectochelle_rereduction/data_ngc6819_2012.0509_ngc6819_may2012_3.0223.h5','r')
gaiaid=2076394173950855424

t = th5[f'{gaiaid}']
t['phot'].keys()

<KeysViewHDF5 ['2MASS_H', '2MASS_J', 'GaiaEDR3_BP', 'GaiaEDR3_G', 'GaiaEDR3_RP', 'PS_g', 'PS_i', 'PS_r', 'PS_y']>

In [7]:
import jax.numpy as jnp
import jax
photmod_a = {'2MASS_H': (15.41381765, 'dtype=float64'), '2MASS_J': (15.74595812, 'dtype=float64')}#, '2MASS_Ks': (15.360152, dtype=float64), 'GaiaDR3_BP': (17.26480991, dtype=float64), 'GaiaDR3_G': (16.89867956, dtype=float64), 'GaiaDR3_RP': (16.35697298, dtype=float64), 'PS_g': (17.32971985, dtype=float64), 'PS_i': (16.7093741, dtype=float64), 'PS_r': (16.88157518, dtype=float64), 'PS_z': (16.65634281, dtype=float64), 'WISE_W1': (15.33907628, dtype=float64), 'WISE_W2': (15.34342511, dtype=float64), 'WISE_W3': (15.31416374, dtype=float64), 'WISE_W4': (15.30770745, dtype=float64)}
photmod_b = {'2MASS_H': (15.76234387, 'dtype=float64'), '2MASS_J': (15.56781334, 'dtype=float64')}#, '2MASS_Ks': (15.360152, dtype=float64), 'GaiaDR3_BP': (17.26480991, dtype=float64), 'GaiaDR3_G': (16.89867956, dtype=float64), 'GaiaDR3_RP': (16.35697298, dtype=float64), 'PS_g': (17.32971985, dtype=float64), 'PS_i': (16.7093741, dtype=float64), 'PS_r': (16.88157518, dtype=float64), 'PS_z': (16.65634281, dtype=float64), 'WISE_W1': (15.33907628, dtype=float64), 'WISE_W2': (15.34342511, dtype=float64), 'WISE_W3': (15.31416374, dtype=float64), 'WISE_W4': (15.30770745, dtype=float64)} 


filtarr = ['2MASS_H', '2MASS_J', 'GaiaDR3_BP', 'GaiaDR3_G', 'GaiaDR3_RP', 'PS_g', 'PS_i', 'PS_r', 'PS_y']

# for i in filtarr:
#     for m_a, m_b in zip(photmod_a[i],photmod_b[i]):
#         print(m_a, m_b)

for ii, xx in enumerate(filtarr):
    try:
        photmod_a[ii] = photmod_a[xx]
        #print(ii, xx)
    except KeyError:
        print(f'failed on {xx}')

print(photmod_a)

# for m_a, m_b in zip(photmod_a,photmod_b):
#     print(photmod_a[m_a], photmod_b[m_b])

failed on GaiaDR3_BP
failed on GaiaDR3_G
failed on GaiaDR3_RP
failed on PS_g
failed on PS_i
failed on PS_r
failed on PS_y
{'2MASS_H': (15.41381765, 'dtype=float64'), '2MASS_J': (15.74595812, 'dtype=float64'), 0: (15.41381765, 'dtype=float64'), 1: (15.74595812, 'dtype=float64')}


In [10]:
for ii, xx in enumerate(filtarr):
    try:
        print(photmod_a[ii])
    except KeyError:
        pass

(15.41381765, 'dtype=float64')
(15.74595812, 'dtype=float64')


In [36]:
# photmod_est = (
#     [-2.5 * jnp.log10( 10.0**(-0.4 * m_a) + 10.0**(-0.4 * m_b) ) for m_a, m_b in zip(photmod_a,photmod_b)]
# )

photmod_est = (
    [(m_a, m_b) for m_a, m_b in zip(photmod_a,photmod_b)]
)
photmod_est

[('2MASS_H', '2MASS_H'), ('2MASS_J', '2MASS_J')]